In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re, string, unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from bs4 import BeautifulSoup
import os
import string
from string import digits
import splitter
import enchant, sys

#checking the current working directory
os.getcwd()

#loading the file
filename='input_data.xlsx'
data=pd.read_excel(filename)

#checking the first five enteries in the file
print("First five enteries of the dataframe are:",data.head(5))

First five enteries of the dataframe are:                Short description  \
0                    login issue   
1                        outlook   
2             cant log in to vpn   
3  unable to access hr_tool page   
4                   skype error    

                                         Description             Caller  \
0  -verified user details.(employee# & manager na...  spxjnwir pjlcoqds   
1  \r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail...  hmjdrvpb komuaywn   
2  \r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail...  eylqgodm ybqkwiam   
3                      unable to access hr_tool page  xbkucsvz gcpydteq   
4                                       skype error   owlgqjme qhcozdfx   

  Assignment group  
0            GRP_0  
1            GRP_0  
2            GRP_0  
3            GRP_0  
4            GRP_0  


In [2]:
#various stats of dataframe
print(data.describe().transpose())

#the Caller column is irrelvant for creating the model
data.drop(["Caller"],axis=1,inplace=True)

#filling in missing values
data['Short description'].fillna(value=' ', inplace=True)
data['Description'].fillna(value=' ', inplace=True)

#data preprocessing

def preprocess(text):
#1)remove html tags    
   soup=BeautifulSoup(text,"html.parser")
   text=soup.get_text(separator="")
    
#2) Remove non-ASCII characters
   encoded_string = text.encode("ascii", "ignore")
   text= encoded_string.decode()
   
#3)lower case    
   text=text.lower()
   text = ' '.join([w for w in text.split()])
#4)remove punctuation       
   text = re.sub(r'[^\w\s]', '',text) 
   
#5)remove whitespaces
   text=" ".join(text.split())
 
#6)remove  digits  
   remove_digits = str.maketrans('', '', digits) 
   text = text.translate(remove_digits) 
    
#7)remove emails   
   text = re.sub(r'\S*@\S*\s?', '', text)
   
#8)remove hyperlinks
   text = re.sub(r'https?:\/\/.*\/\w*','', text)
   
#9)remove other characters   
   text=text.replace("_"," ")
   

   return text   


#the preprocessing is applied on the short description and description columns


data['Short description']=data['Short description'].apply(preprocess)
data['Description']      =data['Description'].apply(preprocess)
#verifying the preprocess on sample data
print("First 20 enteries of the dataframe are:",data.head(20))

                  count unique                top  freq
Short description  8492   7481     password reset    38
Description        8499   7817                the    56
Caller             8500   2950  bpctwhsn kzqsbmtp   810
Assignment group   8500     74              GRP_0  3976
First 20 enteries of the dataframe are:                                     Short description  \
0                                         login issue   
1                                             outlook   
2                                  cant log in to vpn   
3                       unable to access hr tool page   
4                                         skype error   
5      unable to log in to engineering tool and skype   
6   event criticalhostname companycom the value of...   
7   ticket no employment status new nonemployee en...   
8                unable to disable add ins on outlook   
9                           ticket update on inplant    
10  engineering tool says not connected and unable...

In [3]:
#language translation

from google_trans_new import google_translator #google translate api wasnt working so alternate version was downloaded
#https://github.com/lushan88a/google_trans_new  using pip install google_trans_new
translator =google_translator()

detect =  google_translator().detect
 
def translatetoeng(txt):
    if detect(txt)!=  'en':
        translation = translator.translate(txt)
    else:
        translation = txt
    return translation
#THIS PART OF CODE IS JUST FOR TRYING THE FUNCTION STARTING HERE
#trying the detection
x="हिंदी टायपिंग"
detect(x)
#trying the def translatetoeng on a single non english row of the dataframe
print((data.iloc[255]['Description']))
y=data.iloc[255]['Description']
detect(y)#here the output is german
#data.iloc[255]['Description']=data.iloc[255]['Description'].apply(translatenoteng)
y=translatetoeng(y)
print(y)
detect(y)#now the output has been converted to english by the function translatetoeng
#ENDING HERE
#applying the function  on the 2 columns of the dataframe

data['Short description']=data['Short description'].apply(translatetoeng)
data['Description']=data['Description'].apply(translatetoeng)

probleme mit laufwerk z laeusvjo fvaihgpx
problems with drive z laeusvjo fvaihgpx 


In [4]:
#checking random entry to check if it has been translated
y=data.iloc[255]['Description']
print(y)

problems with drive z laeusvjo fvaihgpx 


In [5]:
#Remove non english words
nltk.download('words')
from nltk.corpus import words
Word = list(set(words.words()))
data['Description'] = [" ".join(w for w in nltk.wordpunct_tokenize(x) 
                       if w.lower() in Word or not w.isalpha()) 
                       for x in data['Description']]

#testing one a single entry
print(data.iloc[255]['Description'])


data['Short description'] = [" ".join(w for w in nltk.wordpunct_tokenize(x) 
                       if w.lower() in Word or not w.isalpha()) 
                       for x in data['Short description']]

print(data.iloc[255]['Short description'])


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\vin\anaconda3\lib\nltk_data...
[nltk_data]   Package words is already up-to-date!


with drive z
with drive z


In [6]:
#Remove stop words
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')
data['Short description'] = data['Short description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data['Description'] = data['Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vin\anaconda3\lib\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
#checking one column after stopword removal
print(data['Description'])

0       user manager name checked user name ad reset p...
1       received hello team outlook calendar somebody ...
2                             received hi cannot log best
3                                 unable access tool page
4                                                   error
                              ...                        
8495      received good afternoon sent mail please advise
8496                                      telephony issue
8497                                       password reset
8498    unable access machine finish drawers adjustmen...
8499              Different program cannot several . area
Name: Description, Length: 8500, dtype: object


In [8]:
#merging  the 2 preprocessed columns to a single column without duplicate words
data['Combined description'] = data['Short description'] .map(str) + ' ' +  data['Description'].map(str)
                    
data['Combined description'] = data['Combined description'].apply(lambda x: ' '.join(pd.unique(x.split()))) 
   
#testing on single entry
print(data.iloc[279]['Short description'])
print(data.iloc[279]['Description'])
print(data.iloc[279]['Combined description'])     

unable create print cant post detail information see attachment
please provide following order number material item number warehouse location plant issue description error message see attachment
unable create print cant post detail information see attachment please provide following order number material item warehouse location plant issue description error message


In [9]:
#expand contractions
contractions_dict = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"didnt": "did not",
"doesnt": "does not",
"thats": "that is",
"wasnt": "was not",
"weren": "were not",
"theyre": "there",
"dont": "do not",
"cant": "cannot",
"arent": "are not",
"whats": "what is",
"you've": "you have"
}
# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

# Expanding Contractions in the reviews
data['Combined description']=data['Combined description'].apply(lambda x:expand_contractions(x))

In [10]:
from autocorrect import Speller 
spell =Speller('en')       #Speller(fast=True) for faster but less accurate correctiondata
data['Combined description']=[' '.join([spell(i) for i in x.split()]) for x in data['Combined description']
                             ]

In [89]:
#additional preprocessing
import string
def preprocess(text):
 text=text.replace(" vas "," ")
 text=text.replace(" ae "," ")
 text=text.replace("theres ","there is ")
 text=text.replace("wouldnt ","would not ")
 text=text.replace(" amar "," ")
 text=text.replace(" ted "," ")
 text=text.replace(" sam "," ")
 text=text.replace(" marc "," ")
 text=text.replace(" ross "," ")
 text=text.replace("maint ","maintenance ")
 text=text.replace(" ess"," ")
 text=text.replace("roanoke "," ")
 text=text.replace("futbol "," ")
 text=text.replace(" shi "," ")
 text=text.replace(" pur "," ")
 text=text.replace(" wy "," ")
 text=text.replace(" wy"," ")
 text=text.replace(" ce "," ")
 text=text.replace(" alt "," alternate ")
 text=text.replace(" div "," division ")
 text=text.replace(" tam "," team ")
 text=text.replace("shouldnt "," should not ")
 text=text.replace("haventnt "," have not ")
 text=text.replace("serrata "," ")
 text=text.replace(" ko "," ")
 text=text.replace(" sab "," ")
 text=text.replace(" fe "," ")
 text=text.replace(" pu "," ")
 text=text.replace(" po "," ")
 text=text.replace(" das "," ")
 text=text.replace(" mio "," ")
 text=text.replace(" alfa "," ")
 text=text.replace("squadra "," ")
 text=text.replace(" ora "," ")
 text=text.replace(" nt "," want ")
 text=text.replace(" nt"," want ")
 text=text.replace(" dow"," down")
 text=text.replace(" th "," ")
 text=text.replace(" se "," ")
 text=text.replace(" li "," ")
 text=text.replace(" sal "," ")
 text=text.replace(" os "," ")
 text=text.replace(" u "," ")
 text=text.replace(" bu "," ")
 text=text.replace(" fu "," ")
 text=text.replace(" mein "," ")
 text=text.replace(" ess "," ")
 text=text.replace(" ist "," ")
 text=text.replace(" ewe "," ")
 text=text.replace(" poe "," power ")
 text=text.replace(" rm","warm")
 text=text.replace(" rm ","warn")
 text=text.replace(" lan "," ")
 text=text.replace(" ug "," ")
 text=text.replace(" wa "," ")
 text=text.replace(" wo "," ")
 text=text.replace(" ie"," ")
 text=text.replace(" ie "," ")
 text=text.replace(" st "," ")
 text=text.replace(" ey "," ")
 text=text.replace(" mo "," ")
 text=text.replace(" mir "," ")
 text=text.replace("aw "," ")
 #text=text.replace("ie "," ")
 text=text.replace(" oe "," ")
 text=text.replace(" aa "," ")
 text=text.replace("sa "," ")
 text=text.replace(" si "," ")
 text=text.replace(" das "," ")
 text=text.replace(" dob "," ")
 text=text.replace(" bisch "," ")
 text=text.replace(" iso "," ")
 text=text.replace(" och "," ")
 text=text.replace("sao "," ")
 text=text.replace("eu ","european union ")
 text=text.replace("withdr","withdraw ")
 text=text.replace("serverdown"," server down ")
 text=text.replace("timecard"," time  card ")
 text=text.replace(" rning","warning ")
 text=text.replace("cleanup","clean up ")
 text=text.replace(" vida "," via ")
 text=text.replace(" proto ","  ")
 text=text.replace(" u ","  ")
 text=text.replace(" fi ","  ")
 text=text.replace(" campo "," ")
 text=text.replace(" corp "," ")
 text=text.replace("dup "," ")
 text=text.replace(" ar "," ")
 text=text.replace("brock "," ")
 text=text.replace(" hanna "," ")
 text=text.replace("hanna "," ")
 text=text.replace(" sherlock "," ")
 text=text.replace(" wi fi "," wifi ")
 text=text.replace(" sud "," ")
 text=text.replace(" anyways "," anyway ")
 text=text.replace(" anyways"," anyway ")
 text=text.replace(" bom "," ")
 text=text.replace(" curr "," current ")
 text=text.replace(" center pam"," center ")   
 text=text.replace(" noncompliance"," non compliance")
 text=text.replace(" intercompany "," inter company ")
 text=text.replace(" wi "," wifi ")
 text=text.replace(" herman "," ")
 text=text.replace(" sind "," ")
 text=text.replace(" neal "," ")
 text=text.replace(" awa "," ")
 text=text.replace(" wir "," ")
 text=text.replace(" rita "," ")
 text=text.replace(" didnt "," did not ")
 text=text.replace("ess portal "," portal ") 
 text=text.replace(" whats "," what is ") 
 text=text.replace(" thats "," that is ")
 text=text.replace(" arent "," are not ")
 text=text.replace(" wasnt "," was not ")
 text=text.replace("parlbox","mailbox ")
 text=text.replace("ess login "," login ")
 text=text.replace("ess portal "," portal ")
 text=text.replace("ess expense "," expense ") 
 text=text.replace("ess password "," password ")  
 text=text.replace(" till st"," till ")
 #text=text.replace(" tool st "," tool ")
 text=text.replace("fi two "," two ")
 text=text.replace("ess portal "," portal ") 
 text=text.replace("ess access "," access ")    
 text=text.replace("ess site "," site ") 
 text=text.replace(" login ess"," login ")
 text=text.replace("ko assign "," assign ")  
 text=text.replace(" period til "," period till ") 
 text=text.replace("write ce"," write manager")
 text=text.replace("since th"," since ") 
 #text=text.replace("since st"," since ") 
 text=text.replace("wi access ","wifi access ") 
 text=text.replace(" bis wird"," ") 
 text=text.replace(" used si"," used ") 
 text=text.replace(" possible last th "," possible last ")
 text=text.replace("ey consultant"," consultant ") 
 text=text.replace("approval ce"," approval ") 
 text=text.replace("since st "," since ")
 text=text.replace("pur wrong "," wrong ") 
 text=text.replace(" rie start "," start ") 
 text=text.replace("dob report ","report ")
 text=text.replace(" addr login "," address login ") 
 text=text.replace(" room fe"," room ")
 text=text.replace(" futbol"," football ") 
 text=text.replace(" current st "," current") 
 text=text.replace(" examine nat "," examine ") 
 text=text.replace("synchro "," synchronise ") 
 text=text.replace(" delete th"," delete ") 
 text=text.replace(" global naa "," global ") 
 text=text.replace("mio cannot "," cannot ") 
 text=text.replace("fe does "," does ") 
 text=text.replace(" went colin "," went ") 
 text=text.replace(" user sar r "," user ") 
 text=text.replace(" hi ti "," ") 
 text=text.replace(" sao campo ","  ") 
 text=text.replace(" sao campo","  ") 
 text=text.replace(" havent "," have not ")    
 text=text.replace(" pinto ","  ") 
 text=text.replace(" system st"," system  ") 
 text=text.replace(" guest wirel "," guest wireless  ") 
 text=text.replace(" pax ", " ") 
 text=text.replace(" un kwown "," unknown ") 
 text=text.replace("lan n "," lan ") 
 text=text.replace("ted update "," wanted update  ") 
 text=text.replace(" pam ferreira ","  ") 
 text=text.replace("ora internal "," internal  ") 
 text=text.replace(" empty acc "," empty  ") 
 text=text.replace(" acc "," access ")
 text=text.replace(" ferreira "," ")
 text=text.replace(" show te "," show the  ") 
 text=text.replace(" gen ing "," ") 
 text=text.replace(" cutoff "," cut off  ") 
 text=text.replace(" fill rate th"," fill rate  ") 
 text=text.replace(" printed sie "," printed ") 
 text=text.replace("working campo "," working  ") 
 text=text.replace("working campo"," working  ")    
 text=text.replace(" addr "," ")    
 text=text.replace(" delivery hoi "," delivery ") 
 text=text.replace("unable acc "," unable access ") 
 text=text.replace(" see sny "," see ") 
 text=text.replace("wy printer received "," printer received ") 
 text=text.replace("update several th"," update several ") 
 text=text.replace(" log int access "," log access ")
 text=text.replace(" joe ","  ") 
 text=text.replace(" na ","  ")    
 text=text.replace(" andstill "," and still  ") 
 text=text.replace("ess kiosk "," kiosk ")
 text=text.replace(" need active st"," need active status ") 
 text=text.replace(" tool opped "," tool stopped ")  
 text=text.replace(" assembly sha "," assembly ")
 text=text.replace(" campo"," ") 
 text=text.replace(" since atus "," since status ") 
 text=text.replace(" amy "," ") 
 text=text.replace(" department tha "," department ") 
 text=text.replace(" possible last th"," possible last ") 
 text=text.replace(" retiring company th"," retiring company ")
 text=text.replace(" downnn"," down")
 text=text.replace("intercompany "," inter company ")
 text=text.replace("ana call "," call ") 
 text=text.replace("setup machine wa"," setup machine ")
 text=text.replace("drum south sa"," drum south ")
 text=text.replace("ar since"," since ")
 text=text.replace("wi access "," wifi access ")
 text=text.replace(" extend current st"," extend current status ")
 #text=text.replace(" dialin "," dialing ")    
 return text

data["Combined description"]= data["Combined description"].apply(preprocess)
#drop empty columns(some columns became empty after preprocessing)
nan_value = float("NaN")
data.replace("", nan_value, inplace=True)

#drop the 2 description columns as all the information is there in the combined description columnn
data.drop(["Short description","Description"],axis=1,inplace=True)
#drop other columns with no info
data.drop([2932],axis=0,inplace=True)
data.drop([2848],axis=0,inplace=True)


In [90]:
#pickle data
import pickle         
data.to_pickle('preprocesseddata.pkl')
                          

In [91]:
writer = pd.ExcelWriter('pro_trans_inputv4.xlsx', engine='xlsxwriter')
data.to_excel(writer, sheet_name='Sheet1')
writer.save()                        
                          